In [26]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from tqdm import tqdm
tqdm.pandas()

The plan of action is to use user from [3], book detail from [2], and genre and author from [1].

# Book Details

## How to Merge Books?

One way is to use ISBNs. The problem with this is that the same book (by the same author) can have different ISBN depending on the publisher. Another approach is to use book title along with the author name. Of course, this will result in some more problems.

## Books from [2]

In [27]:
books2 = pd.read_csv('2/book200k-300k.csv')
books2.head()

,Publisher,RatingDistTotal,RatingDist5,PublishDay,Name,Rating,pagesNumber,Language,PublishMonth,Id,RatingDist4,RatingDist1,ISBN,RatingDist2,CountsOfReview,Authors,RatingDist3,PublishYear
0,HarperCollins,total:27,5:9,9,Meet My Staff,3.52,40,NaN,11,200000,4:7,1:4,0060274840,2:3,5,Patricia Marx,3:4,1998
1,HarperCollins,total:29,5:14,10,Now Everybody Really Hates Me,4.10,32,NaN,20,200002,4:7,1:1,0060212934,2:1,3,Jane Read Martin,3:6,1993
2,Andrews McMeel Publishing,total:10,5:4,3,"1,003 Great Things About Moms",4.00,304,NaN,1,200003,4:4,1:1,0740722395,2:0,1,Lisa Birnbach,3:1,2002
3,Alianza,total:525,5:182,6,La habitación cerrada y otros cuentos de terror,3.97,232,spa,30,200007,4:179,1:5,8420672564,2:25,5,H.P. Lovecraft,3:134,2005
4,Alianza Editorial,total:81,5:22,4,La llamada de Cthulhu y otros cuentos,3.98,208,spa,1,200009,4:39,1:1,NaN,2:2,5,H.P. Lovecraft,3:17,2004


In [28]:
books1 = pd.read_csv('1/dataset_.csv')
books1.head()

,authors,categories,format,isbn10,isbn13,lang,publication-date,title
0,[1],"[214, 220, 237, 2646, 2647, 2659, 2660, 2679]",1.0,184018907X,9781840189070,en,2004-10-14 00:00:00,Soldier Five : The Real Truth About The Bravo ...
1,"[2, 3]","[235, 3386]",1.0,184454737X,9781844547371,en,2009-03-13 00:00:00,Underbelly : The Gangland War
2,[4],"[358, 2630, 360, 2632]",1.0,8416327866,9788416327867,es,2020-04-30 00:00:00,"A Sir Phillip, Con Amor"
3,"[5, 6, 7, 8]","[377, 2978, 2980]",1.0,571308996,9780571308996,en,2015-10-01 00:00:00,QI: The Third Book of General Ignorance
4,[9],"[2813, 2980]",2.0,8352518,9780008352516,en,2019-06-18 00:00:00,The Hidden Power of F*cking Up


In [29]:
books1.dtypes

authors              object
categories           object
format              float64
isbn10               object
isbn13                int64
lang                 object
publication-date     object
title                object
dtype: object

In [49]:
import re

In [50]:
def book_number(name):
    regex = re.compile(r".*(#\d{1,2}).*")
    find = regex.findall(name)
    if len(find):
        return find[0]
    else:
        return "NA"


In [51]:
books1_s = books1.sample(frac=0.1)

In [54]:
# books1_s["title"].apply(book_number).value_counts()

In [30]:
books1["isbn13"] = books1["isbn13"].astype(str)

In [31]:
books2.dtypes

Publisher           object
RatingDistTotal     object
RatingDist5         object
PublishDay           int64
Name                object
Rating             float64
pagesNumber          int64
Language            object
PublishMonth         int64
Id                   int64
RatingDist4         object
RatingDist1         object
ISBN                object
RatingDist2         object
CountsOfReview       int64
Authors             object
RatingDist3         object
PublishYear          int64
dtype: object

In [32]:
books2.shape

(56182, 18)

In [33]:
authors1 = pd.read_csv("1/authors.csv")
authors1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 654021 entries, 0 to 654020
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   author_id    654021 non-null  int64 
 1   author_name  654019 non-null  object
dtypes: int64(1), object(1)
memory usage: 10.0+ MB


In [34]:
authors1 = authors1.set_index("author_id").sort_index()

In [35]:
authors1.iloc[10]

author_name    Leonard Mlodinow
Name: 11, dtype: object

In [36]:
merged =pd.merge(books2, books1, how='inner', right_on="title", left_on="Name")

In [37]:
merged.sample(5).T

,13766,9639,3213,4990,15031
Publisher,Clarendon Press,Pleasant Word,Corgi,Penguin Classics,"Oxford University Press, USA"
RatingDistTotal,total:7269,total:1,total:943,total:45820,total:907620
RatingDist5,5:1131,5:0,5:164,5:19908,5:357788
PublishDay,3,12,5,8,1
Name,Troilus and Cressida,Evensong,(Un)arranged Marriage,A Hero of Our Time,The Picture of Dorian Gray
Rating,3.37,4.0,3.35,4.11,4.08
pagesNumber,360,476,272,185,249
Language,NaN,NaN,eng,eng,NaN
PublishMonth,23,17,3,30,1
Id,286679,254744,217253,226378,297863


In [38]:
merged.iloc[10]["authors"]

'[887, 50967]'

In [39]:
def match_author_name(ids):
    ids = eval(ids)
    authors = []
    for id_ in ids:
        author = authors1.iloc[id_-1]["author_name"]
        authors.append(author)
    return ", ".join(authors)

In [40]:
merged['authors'] = merged['authors'].progress_apply(match_author_name)

100%|██████████| 15266/15266 [00:02<00:00, 6442.38it/s]


In [41]:
merged.sample(5).T

,2959,6295,11914,3387,8337
Publisher,Marcel Dekker,Princeton University Press,Cornerstone,Pocket Books,Princeton University Press
RatingDistTotal,total:5,total:94,total:64497,total:494,total:2737
RatingDist5,5:0,5:35,5:24342,5:111,5:932
PublishDay,4,7,7,9,3
Name,The Three Little Pigs,Democracy and Tradition,Blindsighted,Sanctuary,The Conscience of a Conservative
Rating,3.4,4.14,4.09,3.62,3.88
pagesNumber,32,368,416,273,144
Language,NaN,NaN,eng,NaN,eng
PublishMonth,1,25,31,1,1
Id,215126,232477,272222,217582,244909


In [17]:
rating = pd.read_csv("3/Ratings.csv")
rating.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [20]:
rating.shape, merged.shape

((1149780, 3), (15266, 26))

In [21]:
merged_2 = pd.merge(merged, rating, on="ISBN", how="inner")
merged_2.shape

(43208, 28)

In [23]:
merged["ISBN"].nunique(), merged_2["ISBN"].nunique()

(5640, 1182)

In [22]:
merged_2.sample(5).T

,4976,9466,26985,8755,41570
Publisher,Pocket Books,Atlantic Monthly Press,Bantam,Bantam Classics,Warner Books
RatingDistTotal,total:708,total:73868,total:4675,total:125011,total:164543
RatingDist5,5:166,5:26873,5:1129,5:44590,5:68504
PublishDay,12,8,9,4,8
Name,The Captain's Daughter,Peace Like a River,Couplehood,The Woman in White,The Rescue
Rating,3.82,4.0,3.72,4.0,4.11
pagesNumber,278,312,224,783,420
Language,eng,eng,eng,eng,en-US
PublishMonth,1,7,1,1,1
Id,217607,227571,232336,227524,289110


In [44]:
merged_3 = pd.merge(books2, rating, on = "ISBN", how="inner")
merged_3.head().T

,0,1,2,3,4
Publisher,Alianza Editorial Sa,University of New Mexico Press,University of New Mexico Press,University of New Mexico Press,University of New Mexico Press
RatingDistTotal,total:4881,total:1528,total:1528,total:1528,total:1528
RatingDist5,5:2285,5:604,5:604,5:604,5:604
PublishDay,6,1,1,1,1
Name,Dagón y otros cuentos macabros,Wisconsin Death Trip,Wisconsin Death Trip,Wisconsin Death Trip,Wisconsin Death Trip
Rating,4.21,4.03,4.03,4.03,4.03
pagesNumber,257,264,264,264,264
Language,spa,eng,eng,eng,eng
PublishMonth,30,1,1,1,1
Id,200010,200081,200081,200081,200081


In [46]:
merged["ISBN"].nunique(), merged_2["ISBN"].nunique(), merged_3["ISBN"].nunique(), books2["ISBN"].nunique()

(5640, 1182, 7433, 55899)

In [48]:
merged.shape, merged_2.shape, merged_3.shape, books2.shape

((15266, 26), (43208, 28), (52085, 20), (56182, 18))